In [0]:
#Import Library

import pandas as pd
import numpy as np
import re
import nltk


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Analisis karakteristik Data 

In [3]:
data = pd.read_excel(r"/content/drive/My Drive/Dataset/SentimentDatasetonAppReviewfromAppStore.xlsx")
data.head() #Menampilkan 5 Data Teratas


,ID,TITLE,COMMENT,RATING,SENTIMEN
0,1611,Banyak kekurangan,"Tidak seperti di iphone menunya kurang, jadwal...",1,Negative
1,1612,Need update!,Kurang bersahabat ama VoiceOver nih. Semoga ha...,4,Negative
2,1613,Baru seperti ini sudah puas?,"Hampir setahun belum ada update, UI tidak ada ...",1,Negative
3,1614,Crash melulu,Mohon appnya diperbaiki.. Crashnya ga ketolong...,1,Negative
4,1615,Sampah,"Aplikasi sampah, crash mulu,errorny banyak,ga ...",1,Negative


In [4]:
#Melihat deskripsi data

data.describe()


,ID,RATING
count,553.000000,553.000000
mean,2268.273056,3.086799
std,342.352755,1.747583
min,1611.000000,1.000000
25%,2044.000000,1.000000
50%,2279.000000,3.000000
75%,2586.000000,5.000000
max,2761.000000,5.000000


In [5]:
#Melihat banyak baris dan kolo,

data.shape

(553, 5)

In [6]:
#Melihat jumlah pada setiap label
data['SENTIMEN'].value_counts()


Positive    259
Negative    241
Neutral      53
Name: SENTIMEN, dtype: int64

Disini kita mengetahui berapa saja jumlah komentar yang positif, negative,da neutral

#Preproccessing data

In [0]:
#Replace label "negative" menjadi "Negative" menggunakan pandas

#data['label']= data['label'].replace("negative","Negative")
#data['label'].value_counts()

##Create Train & Test Using Scikitlearn

Memisahkan data Train dan Test menggunakan lIbrary sklearn - test


In [0]:
from sklearn.model_selection import train_test_split


In [0]:
#Inisialisasi Variable x dan y yang akan di proses
x = data.COMMENT
y = data.SENTIMEN


Setelah inisialisasi Variable, selanjtnya memisahkan x_train, y_train, x_test, y_test yang digunakan untuk membuat model. train_test_split digunakan untuk memisahkan variable (x,y dengan pembagi data 80%(train:20%(test) sehingga nilai test_size adalah 0.2))


In [0]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=225)

setelah melakukan pemisahan menggunakan train_test_split, kita dapat menampilkan hasil pembagian masing-masing data


In [11]:
#Menampilkan Jumlah data x_train,y_train, x_test,y_test.
print('Jumlah Data x_train : ',len(x_train))
print('Jumlah Data y_train : ',len(y_train))
print('Jumlah Data x_test : ',len(x_test))
print('Jumlah Data y_test : ',len(y_test))

Jumlah Data x_train :  442
Jumlah Data y_train :  442
Jumlah Data x_test :  111
Jumlah Data y_test :  111


##Cleaning Data Text

###Pembuatan Function Clean Data

Untuk membersihkan data, kita dapat menggunakan stopword dari Sastrawi, yaitu stopword bahasa Indonesia untuk menghilangkan stopword dari data yang kita miliki.
dan kita dapat menggunakan wordPunctTokenizer untuk menghilangkan data berupa karakter serti "!,?,@,$." dll

###Menghilangkan Stopword dan Steem Menggunakan Sastrawi
(Menjadikan kata pada "Text" menjadi kata Dasar)

In [12]:
import re
from nltk.tokenize import WordPunctTokenizer
#Install Sastrawi
!pip install Sastrawi

In [0]:
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from tqdm import tqdm_notebook as tqdm

# create stemmer
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

#Create STOP Word Remover
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()


#Function untuk membersihkan data text
def clean_text(list_of_text):
  output_text = []
  for text in tqdm(list_of_text):    
    text = text.translate(str.maketrans("","", string.punctuation)).lower()  
    text =  re.sub(r'[^a-zA-Z0-9]', ' ', text)
    text = stopword.remove(text)
    text = stemmer.stem(text)
    output_text.append(text)
  
  return output_text

**Function** clean_text data terdiri dari :

punct = Untuk menghilangkan Punctuation seperti simbol, titik, koma, tanda tanya dll dan menjadikan semua huruf menjadi kecil

text = memastikan bahwa object, only ASCII A-Za-z0-*9*

text = menghilangkan stopword berbahasa indonesia menggunakan sastrawi

text = stemmer untuk menjadikan kata pada data menjadi kata dasar, exp : Perbanyak = Banyak

**Hasil** text yang sudah dibersihkan akan dimasukkan kedalam vaiable **output_text**

**tqdm** adalah library yang dapat menampilkan progress bar dan waktu proses


In [14]:
results = clean_text(list(data["COMMENT"]))


In [15]:
print(results)

['seperti iphone menu kurang jadwal sepak bola tanggal 2013 baik', 'kurang sahabat ama voiceover nih moga ini jadi perhati update ikut sekali dong tambahin menu buat contact developer nya', 'hampir tahun ada update ui ada kembang fitur tambah crash ada baik rate 1 star uninstall', 'mohon appnya baik crashnya ga tolong', 'aplikasi sampah crash muluerrorny banyakga dibenerin', 'muat berita yg tak bobot butuh user interface yg murah sangat selaras', 'sekarang crash melulu kerja fix this please', 'selalu berita terpotonguntuk ipadiphoneataupun blackberrydan harus browsinglalu apa aplikasi', 'thanks for making this great ebook for indonesian keep it up detik', 'bagus sayang buggy gk ketulungan', 'thank udah yg versi ipad hehheheheheh', 'kok detikbola ga di detik for ipad', 'asslamuallaikum benar aplikasi bagus banget manfaat seluruh baik masih beberapa yg baik contoh akan mendownload audio selalu crashdan ukur huruf besar cara di pinchmdh2an depan bisa baik bug tsbsemoga seluruh tim senanti

##Trial

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [0]:
cvec = CountVectorizer(lowercase=True)
tfvec = TfidfVectorizer()
hvec = HashingVectorizer()

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn import svm

In [0]:
#inisialisasi Algoritma classifikasi yang digunakan
clf_rfc = RandomForestClassifier() 
clf_lr = LogisticRegression()
clf_bnb = BernoulliNB()
clf_svm = svm.SVC(gamma='scale')
clf_gnb = GaussianNB()

##Membuat Pipeline dengan beberapa perbandingan Algoritma

Algoritma yang menjadi percobaan adalah algoritma 
1. LogisticRegression()
2. RandomForestClassifier()
3. BernoulliNB()
4. SVM()
5. GaussianNB()

In [0]:
from sklearn.pipeline import Pipeline

In [0]:
#membuat pipeline LogisticRegression()
model_lr = Pipeline([('tfidf',tfvec),
                    ('clasifier', clf_lr)])

#membuat pipeline RandomForestClassifier()
model_rfc = Pipeline([('tfidf', tfvec),
                 ('classifier', clf_rfc)])

#membuat pipeline BernoulliNB()
model_bnb = Pipeline([('tfidf', tfvec),
                 ('classifier', clf_bnb)])

#membuat pipeline SVM()
model_svm = Pipeline([('tfidf', tfvec),
                 ('classifier', clf_svm)])

#membuat pipeline GaussianNB()
model_gnb = Pipeline([('tfidf', tfvec),
                 ('classifier', clf_gnb)])

In [22]:
#model_lr.fit(list(x_train),list(y_train))
model_lr.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clasifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_s

In [23]:
#Train data X dan y menggunakan RandomForestClassifier()
model_rfc.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                               

In [24]:
#Train data X dan y menggunakan BernoulliNB()
model_bnb.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None,
                             fit_prior=True))],
         verbose=Fal

In [25]:
#model_svm.fit(list(x_train),list(y_train))
model_svm.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamm

In [26]:
print(list(x_train)[10])

Masih ada bugs saat hendak merubah tgl lahir di profile tidak bisa. Mohon di update


##Membuat Class Predict untuk mengetahui hasil dari train menggunakan x_test

In [0]:
hasil_lr = model_lr.predict(x_test)

In [0]:

hasil_svm = model_svm.predict(x_test)

In [0]:
hasil_rfc = model_rfc.predict(x_test)

In [0]:
hasil_bnb = model_bnb.predict(x_test)

##Melihat hasil confusion matriks dari setiap algoritma

In [0]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [32]:
#Confusion Matrix RandomForestClassifier()
confusion_matrix(hasil_rfc,y_test)

array([[25,  4, 21],
       [ 2,  0,  0],
       [12,  5, 42]])

In [33]:
#Confusion Matrix RandomForestClassifier()
confusion_matrix(hasil_rfc,y_test)

array([[25,  4, 21],
       [ 2,  0,  0],
       [12,  5, 42]])

In [34]:
#Confusion Matrix LogisticRegression()
confusion_matrix(hasil_lr,y_test)

array([[32,  7, 16],
       [ 0,  0,  0],
       [ 7,  2, 47]])

In [35]:
#Confusion Matrix BernoulliNB()
confusion_matrix(hasil_bnb,y_test)

array([[22,  4, 11],
       [ 0,  0,  0],
       [17,  5, 52]])

In [36]:
#Confusion Matrix SVM()
confusion_matrix(hasil_svm,y_test)

array([[33,  7, 17],
       [ 0,  0,  0],
       [ 6,  2, 46]])

##Melihat Hasil Akurasi Score

In [37]:
#Accuracy Score LogisticRegression()
accuracy_score(hasil_lr,y_test)

0.7117117117117117

In [38]:
#Accuracy Score RandomForestClassifier()
accuracy_score(hasil_rfc, y_test)

0.6036036036036037

In [39]:
#Accuracy Score BernoulliNB()
accuracy_score(hasil_bnb, y_test)

0.6666666666666666

In [40]:
#Accuracy Score svm()
accuracy_score(hasil_svm,y_test)

0.7117117117117117